날씨 온도 예측, 종속변수 :actual(최고온도)  
데이터 출처 : https://towardsdatascience.com/random-forest-in-python-24d0893d51c0  
데이터 경로 : /kaggle/input/adp-kr-p2/problem1.csv  
temp_1 : 전날 최고온도  
temp_2 : 전전날 최고온도  
friend : 친구의 예측온도  
<br>
###1-1) 데이터 확인 및 전처리  
- 데이터 EDA 수행  
- 결측치를 확인하고 처리 방안에 대해 논의하라  
- 데이터 분할 방법 설명  
- 최종 데이터셋이 적절함을 주장하라  

###1-2) Random Forest 모델 적합 및 검증  
- 예측 기준선 수립 및 근거 제시  
- Random Forest 학습 및 예측 결과 해석  
- 예측 결과 검정 해석, 중요변수 도출  
- 변수 중요성 분석 및 그래프 출력  

###1-3) SVM(Support Vector Machine) 모델 적합 및 검증  
- 예측 기준선 수립 및 근거 제시  
- Random Forest 학습 및 예측 결과 해석  
- 예측 결과 검정 해석, 중요변수 도출  
- 변수 중요성 분석 및 그래프 출력  

###1-4) 모델 비교 및 향후 개선 방향 도출
- Random Forest, SVM 모델의 결과 비교 후 최종 모델 선택
- 두 모델의 장단점 분석, 추후 운영 관점에서 어떤 모델을 선택할 것인가?
- 모델링 관련 추후 개선 방향 제시

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df1 = pd.read_csv('/content/drive/MyDrive/data/adp/week12_기출2/problem1.csv')
df1.head()

,year,month,day,week,temp_2,temp_1,average,actual,forecast_noaa,forecast_acc,forecast_under,friend
0,2016,1,1,Fri,45,45,45.6,45,43,50,44,29
1,2016,1,2,Sat,44,45,45.7,44,41,50,44,61
2,2016,1,3,Sun,45,44,45.8,41,43,46,47,56
3,2016,1,4,Mon,44,41,45.9,40,44,48,46,53
4,2016,1,5,Tues,41,40,46.0,44,46,46,46,41
